# Calibration of the Average Oracle eigenvalues

2021-10-20

author: Christian Bongiorno (christian.bongiorno@centralesupelec.fr)

2023-01

Adapted by Pau Autrand and Thomas Bienaimé for this project

Note: the computation takes about a 2s.

In [1]:
import os
# To speed up multiprocessing. Force numpy to work on a single thread
os.environ["OMP_NUM_THREADS"] = "1"
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
def get_sortest_eig(C):
    '''
    input 
        C: correlation matrix
        
    output: 
        l: eigenvalues
        v: eigenvectors 
    '''
    
    l,v = np.linalg.eigh(C)
    ordn = np.argsort(l)
    l,v = l[ordn],v[:,ordn]
    return l,v

In [4]:
def selectData(t,dtin,dtout):
    '''
    input 
        t: index of today
        dtin: in-sample window size
        dtout: out-of-sample window size
    output
        cin: in-sample correlation
        cout: out-sample correlation
    
    '''
    
    c = C[:,:,t-dtin:t+dtout]

    #Split in-sample and out-of-sample
    cin,cout = c[:,:,:dtin],c[:,:,dtin:]

    return cin,cout


In [5]:
def get_Oracle(t):
    '''
    input
        t: index of today


    output:
        oracle eigenvalues
    '''
    

    #get in-sample and out-of-sample
    cin,cout = selectData(t, 1,1)
    
    #compute correlation matrices
    Cin,Cout = cin.mean(axis= 2),cout.mean(axis= 2)

    #get eigenvalues and eigenvectos
    l,v = get_sortest_eig(Cin)

    #compute oracle
    oracle = (v.T @ Cout @ v).diagonal()
    return oracle

In [6]:

def compute_AO(dtin,dtout,nbr_day, ncpu=None):


    #Available days for the calibratios
    avail_t = np.arange(dtin,Tmax-dtout)

    # selection of all the inputs
    conf_input = [t for t in avail_t]

    p = Pool(processes=ncpu)
    AO = p.map(get_Oracle,conf_input,chunksize=1000)
    p.close()
    
    AO = [i for i in AO if not i is None]

    # Average-Oracle eigenvalues
    AO = np.mean(AO,axis=0)

    print(AO)
    return AO

# you can now save the AO values

In [7]:
# Total number of days
last_year_start_index = 257# from our data, we separated calibration for average oracle and use of it (4 year calibration, 1 year use (2022))
with open('../data/allstocks_correlation.pickle', 'rb') as f:
    C = pickle.load(f)[:,:,:-last_year_start_index]
N=C.shape[0]
Tmax = C.shape[2]
Tmax

1019

In [8]:
days_estimate = [1, 10, 50] #how many days you want to use in order to obtain your covariance estimates.

In [9]:
dtout=1

for nbr_day in days_estimate:
    dtin=nbr_day
    file_AO= f"../data/average_oracle_values/AO_Din{nbr_day}_N{N}-2018-2021.csv"
    AO = compute_AO(dtin,dtout, nbr_day)
    pd.DataFrame( AO )\
        .to_csv(file_AO, index=False, header=False)

[0.82811831 0.81192621 0.8090779  0.81713969 0.81421078 0.82418093
 0.83661522 0.84327151 0.84745787 0.85149157 0.85545996 0.86281456
 0.87147842 0.87446097 0.88133082 0.88613263 0.8954639  0.89570093
 0.90710737 0.91574779 0.92347776 0.92986317 0.93486046 0.9468546
 0.94796196 0.95839305 0.9610648  0.97336491 0.98597705 1.01499127
 1.0444633  1.17384849 4.07569187]
[0.82825783 0.81192369 0.80919569 0.81732214 0.81420978 0.82423589
 0.83674442 0.84342691 0.84762038 0.85176195 0.8556565  0.86259842
 0.87195838 0.87440796 0.88155225 0.88628833 0.89553533 0.89584442
 0.90691145 0.91570988 0.92326839 0.9299243  0.93475916 0.94656743
 0.94797712 0.95792969 0.961128   0.97325426 0.98583411 1.01469276
 1.0446784  1.17419168 4.07463311]
[0.83033449 0.81386693 0.81008185 0.82039903 0.81605097 0.82639341
 0.83839596 0.84605454 0.85156208 0.8526504  0.85885276 0.86546745
 0.87430952 0.8762352  0.8821206  0.88781856 0.89631268 0.89786632
 0.90927707 0.91751132 0.92420959 0.93170493 0.93663254 0.94